In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_4_Water_Network/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 19:41:50.790296


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:

* \(t\) - time step index, \(t = 1, 2, ..., 10\)
* \(f(t)\) - water inflow at point A at time step \(t\)
* \(C_{ij}\) - capacity of pipe connecting points \(i\) and \(j\) (where \(i, j \in \{A, B, C, D, E, F, G\}\) and \(i \neq j\))
* \(N\) - total number of pipes, \(N = 12\)
* \(K\) - maximum number of pipes that can be activated, \(K = 10\)

Decision variables:

* \(x_{ijt}\) - amount of water flowing from point \(i\) to point \(j\) at time step \(t\)
* \(y_{ij}\) - binary variable indicating whether pipe \(ij\) is activated or not (\(y_{ij} = 1\) if activated, \(y_{ij} = 0\) otherwise)

Objective function:

Maximize the total water flow to point G over the 10 time steps:

\(max \sum_{t=1}^{10} x_{Gt}\)

Constraints:

1. Water inflow at point A:
\[x_{A1} = f(1)\]
\[x_{At} = f(t) + \sum_{i \in \{B, C, F\}} x_{it-1}, \forall t = 2, 3, ..., 10\]
2. Water balance at each point (excluding A and G):
\[\sum_{i \in \{A, B, C, F\}} x_{it} = \sum_{j \in \{B, C, D, E, F\}} x_{tj}, \forall t = 1, 2, ..., 10\]
3. Pipe capacity constraints:
\[x_{ijt} \leq C_{ij} y_{ij}, \forall i, j \in \{A, B, C, D, E, F, G\}, i \neq j, t = 1, 2, ..., 10\]
4. Maximum number of activated pipes:
\[\sum_{i, j \in \{A, B, C, D, E, F, G\}, i \neq j} y_{ij} \leq K\]
5. Non-negativity and binary constraints:
\[x_{ijt} \geq 0, \forall i, j \in \{A, B, C, D, E, F, G\}, i \neq j, t = 1, 2, ..., 10\]
\[y_{ij} \in \{0, 1\}, \forall i, j \in \{A, B, C, D, E, F, G\}, i \neq j\]

This is a mixed-integer linear programming (MILP) problem, which can be solved using optimization solvers such as CPLEX, Gurobi, or GLPK.

In [6]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:

* \(t\) - time step index, \(t = 1, 2, ..., 10\)
* \(f(t)\) - water inflow at point A at time step \(t\)
* \(C_{ij}\) - capacity of pipe connecting points \(i\) and \(j\) (where \(i, j \in \{A, B, C, D, E, F, G\}\) and \(i \neq j\))
* \(N\) - total number of pipes, \(N = 12\)
* \(K\) - maximum number of pipes that can be activated, \(K = 10\)

Decision variables:

* \(x_{ijt}\) - amount of water flowing from point \(i\) to point \(j\) at time step \(t\)
* \(y_{ij}\) - binary variable indicating whether pipe \(ij\) is activated or not (\(y_{ij} = 1\) if activated, \(y_{ij} = 0\) otherwise)

Objective function:

Maximize the total water flow to point G over the 10 time steps:

\(max \sum_{t=1}^{10} x_{Gt}\)

Constraints:

1. Water inflow at point A:
\[x_{A1} = f(1)\]
\[x_{At} = f(t) + \sum_{i \in \{B, C, F\}} x_{

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Sample data (You can replace this with actual data)
T = 10
f = {1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90, 10: 100}
C = {('A', 'B'): 50, ('A', 'C'): 50, ('A', 'F'): 50, ('B', 'A'): 50, ('B', 'D'): 50, ('B', 'E'): 50, ('C', 'A'): 50, ('C', 'D'): 50, ('D', 'C'): 50, ('D', 'E'): 50, ('D', 'G'): 50, ('E', 'B'): 50, ('E', 'D'): 50, ('G', 'D'): 50}
N = len(C)
K = 10

# Define the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, T+1), range(1, N+1), domain=pyo.NonNegativeReals)
model.y = pyo.Var(range(1, N+1), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[t, N] for t in range(1, T+1))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def inflow_A_rule(model, t):
    if t == 1:
        return model.x[t, 1] == f[t]
    else:
        return model.x[t, 1] == f[t] + sum(model.x[t-1, i] for i in [2, 3, 6])
model.inflow_A = pyo.Constraint(range(1, T+1), rule=inflow_A_rule)

def water_balance_rule(model, t):
    return sum(model.x[t, i] for i in [2, 3, 6]) == sum(model.x[t, j] for j in [2, 3, 4, 5, 6])
model.water_balance = pyo.Constraint(range(1, T+1), rule=water_balance_rule)

def pipe_capacity_rule(model, i, j, t):
    return model.x[t, i] <= C[(i, j)] * model.y[i]
model.pipe_capacity = pyo.Constraint(range(1, N+1), range(1, N+1), range(1, T+1), rule=pipe_capacity_rule)

def max_pipes_rule(model):
    return sum(model.y[i] for i in range(1, N+1)) <= K
model.max_pipes = pyo.Constraint(rule=max_pipes_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Optimal solution found!")
print("Total water flow to point G: ", pyo.value(model.obj))
print("Activated pipes: ", [i for i in range(1, N+1) if pyo.value(model.y[i]) == 1])
```

This code uses the GLPK solver to solve the optimization problem. You can replace the sample data with actual data and use a different solver if needed.

In [10]:
print(response_2_text)

Here is a Python Pyomo code for the optimization problem:

```python
import pyomo.environ as pyo

# Sample data (You can replace this with actual data)
T = 10
f = {1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90, 10: 100}
C = {('A', 'B'): 50, ('A', 'C'): 50, ('A', 'F'): 50, ('B', 'A'): 50, ('B', 'D'): 50, ('B', 'E'): 50, ('C', 'A'): 50, ('C', 'D'): 50, ('D', 'C'): 50, ('D', 'E'): 50, ('D', 'G'): 50, ('E', 'B'): 50, ('E', 'D'): 50, ('G', 'D'): 50}
N = len(C)
K = 10

# Define the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, T+1), range(1, N+1), domain=pyo.NonNegativeReals)
model.y = pyo.Var(range(1, N+1), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[t, N] for t in range(1, T+1))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def inflow_A_rule(model, t):
    if t == 1:
        return model.x[t, 1] == f[t]
    else:
        return m

### Code Executability

In [11]:
import pyomo.environ as pyo

# Sample data (You can replace this with actual data)
T = 10
f = {1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90, 10: 100}
C = {('A', 'B'): 50, ('A', 'C'): 50, ('A', 'F'): 50, ('B', 'A'): 50, ('B', 'D'): 50, ('B', 'E'): 50, ('C', 'A'): 50, ('C', 'D'): 50, ('D', 'C'): 50, ('D', 'E'): 50, ('D', 'G'): 50, ('E', 'B'): 50, ('E', 'D'): 50, ('G', 'D'): 50}
N = len(C)
K = 10

# Define the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, T+1), range(1, N+1), domain=pyo.NonNegativeReals)
model.y = pyo.Var(range(1, N+1), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[t, N] for t in range(1, T+1))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def inflow_A_rule(model, t):
    if t == 1:
        return model.x[t, 1] == f[t]
    else:
        return model.x[t, 1] == f[t] + sum(model.x[t-1, i] for i in [2, 3, 6])
model.inflow_A = pyo.Constraint(range(1, T+1), rule=inflow_A_rule)

def water_balance_rule(model, t):
    return sum(model.x[t, i] for i in [2, 3, 6]) == sum(model.x[t, j] for j in [2, 3, 4, 5, 6])
model.water_balance = pyo.Constraint(range(1, T+1), rule=water_balance_rule)

def pipe_capacity_rule(model, i, j, t):
    return model.x[t, i] <= C[(i, j)] * model.y[i]
model.pipe_capacity = pyo.Constraint(range(1, N+1), range(1, N+1), range(1, T+1), rule=pipe_capacity_rule)

def max_pipes_rule(model):
    return sum(model.y[i] for i in range(1, N+1)) <= K
model.max_pipes = pyo.Constraint(rule=max_pipes_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Optimal solution found!")
print("Total water flow to point G: ", pyo.value(model.obj))
print("Activated pipes: ", [i for i in range(1, N+1) if pyo.value(model.y[i]) == 1])

2024-05-22 19:45:57,738 ERROR pyomo.core: Rule failed when generating expression for Constraint pipe_capacity with index (1, 1, 1):
KeyError: (1, 1)
2024-05-22 19:45:57,739 ERROR pyomo.core: Constructing component 'pipe_capacity' from data=None failed:
KeyError: (1, 1)


KeyError: (1, 1)

### Solution Correctness

In [12]:
import pyomo.environ as pyo

# Sample data (You can replace this with actual data)
T = 10
f = {1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90, 10: 100}
C = {('A', 'B'): 50, ('A', 'C'): 50, ('A', 'F'): 50, ('B', 'A'): 50, ('B', 'D'): 50, ('B', 'E'): 50, ('C', 'A'): 50, ('C', 'D'): 50, ('D', 'C'): 50, ('D', 'E'): 50, ('D', 'G'): 50, ('E', 'B'): 50, ('E', 'D'): 50, ('G', 'D'): 50}
N = len(C)
K = 10

# Define the Pyomo model
model = pyo.ConcreteModel()

# Define the decision variables
model.x = pyo.Var(range(1, T+1), range(1, N+1), domain=pyo.NonNegativeReals)
model.y = pyo.Var(range(1, N+1), domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[t, N] for t in range(1, T+1))
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def inflow_A_rule(model, t):
    if t == 1:
        return model.x[t, 1] == f[t]
    else:
        return model.x[t, 1] == f[t] + sum(model.x[t-1, i] for i in [2, 3, 6])
model.inflow_A = pyo.Constraint(range(1, T+1), rule=inflow_A_rule)

def water_balance_rule(model, t):
    return sum(model.x[t, i] for i in [2, 3, 6]) == sum(model.x[t, j] for j in [2, 3, 4, 5, 6])
model.water_balance = pyo.Constraint(range(1, T+1), rule=water_balance_rule)

def pipe_capacity_rule(model, i, j, t):
    return model.x[t, i] <= C[(i, j)] * model.y[i]
model.pipe_capacity = pyo.Constraint(range(1, N+1), range(1, N+1), range(1, T+1), rule=pipe_capacity_rule)

def max_pipes_rule(model):
    return sum(model.y[i] for i in range(1, N+1)) <= K
model.max_pipes = pyo.Constraint(rule=max_pipes_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the solution
print("Optimal solution found!")
print("Total water flow to point G: ", pyo.value(model.obj))
print("Activated pipes: ", [i for i in range(1, N+1) if pyo.value(model.y[i]) == 1])

2024-05-22 19:46:28,409 ERROR pyomo.core: Rule failed when generating expression for Constraint pipe_capacity with index (1, 1, 1):
KeyError: (1, 1)
2024-05-22 19:46:28,410 ERROR pyomo.core: Constructing component 'pipe_capacity' from data=None failed:
KeyError: (1, 1)


KeyError: (1, 1)